In [168]:
import numpy as np
import sounddevice as sd
import pyaudio
import math
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [169]:
PI = np.pi

In [196]:
def sin_table(fsin, rate):
    x = np.arange(0, rate/fsin)
    return np.sin(x*2*PI*fsin/rate)

SIN_TABLE = sin_table(fsin, rate)

In [197]:
def robot_voice(samples, index_offset, prev_val, table):
    """
    Must return 3 things :
        - the processed array for robot voice
        - the next index to be used for the lookup table
        - the last value still 'unprocessed'
    """
    last_val = samples[-1] # For the robot_voice() call of the next chunk
    N = len(table)
    
    # Remove the DC component
    samples[0] = samples[0] - prev_val
    samples[1:] = samples[1:] - samples[0:-1]
    
    k = index_offset
    
    for i in range(len(samples)):
        samples[i] = samples[i] * table[k]
        k = (k+1) % N
        
    
    
    
    return samples, k, last_val

In [202]:
# Parameters
rate = 16000 # Hz
fsin = 600 # Hz
CHUNK = 2**12 # samples. The higher the value, the larger the delay but the better the quality
LEN = 10 # to change the duration of the simulation

In [203]:
p = pyaudio.PyAudio()

stream = p.open(format=pyaudio.paInt16, channels=1, rate=rate, input=True, frames_per_buffer=CHUNK)
player = p.open(format=pyaudio.paInt16, channels=1, rate=rate, output=True, frames_per_buffer=CHUNK)

index = 0
prev_value = 0


for i in range(int(LEN*RATE/CHUNK)):
    
    # Input
    data = -np.fromstring(stream.read(CHUNK),dtype=np.int16)
    
    # Processing
    data, index, prev_value = robot_voice(data, index, prev_value, SIN_TABLE) 
    
    # Output
    player.write(data,CHUNK)


stream.stop_stream()
stream.close()
p.terminate()

KeyboardInterrupt: 